## Calculating the required supply for roads with no chargers

In [67]:
import pandas as pd
import numpy as np
import math

# Display all columns in the df
pd.set_option('display.max_columns', None)
# Set the float format to display numbers in standard decimal notation
pd.options.display.float_format = '{:.5f}'.format

In [68]:
mainframe=pd.read_csv("mainframe.csv")

mainframe=mainframe.loc[mainframe["CS_id"]== 0]

### Read in the hourly ratios of traffic

In [69]:
#Investigating real traffic hourly patterns of the day based on real data (not pivoted spreadsheet)
hourly=pd.read_csv(r"/Users/antonglad/Desktop/Python_Recreation/completeAllHourlyVolume_EasterAprJulSep.csv")

# Group the DataFrame by hour and sum the traffic volume
total_traffic_by_hour = hourly.groupby(['from_hour'])['volume'].sum()

# Calculate the total traffic volume for the entire day
total_traffic = hourly['volume'].sum()

# Calculate the mean ratio for each hour of the day
mean_ratios = total_traffic_by_hour / total_traffic

In [70]:
#New dataframe creation which excludes duplicates rows (results in as many rows as edges)
every_edge_without= mainframe.drop_duplicates(subset='new_id', keep='first')

### Create hourly demand based on volume data of roads

In [71]:
# Extract hour ratios from mean_ratios
hour_ratios = mean_ratios.values.tolist()


# Prepare a list of month abbreviations
months = ['jan', 'feb', 'mar', 'apr', 'may', 'jun', 'jul', 'aug', 'sep', 'oct', 'nov', 'dec']

# Initialize an empty dictionary to store the DataFrames for each month
monthly_dfs = {}

# Iterate over each month
for month in months:
    # Create a new DataFrame with the same index as new_df
    monthly_df = every_edge_without[['u', 'v']].copy()
    
    # Distribute cars for each edge and each hour of the day
    for hour, ratio in enumerate(hour_ratios):
        hourly_column_name = f'hour_{hour:02d}_{month}'
        monthly_df[hourly_column_name] = every_edge_without[f'charge_cars_{month}'].mul(ratio)

        monthly_df["max_hourly_value"] = monthly_df.drop(columns=['u', 'v']).max(axis=1)

    
    # Store the resulting DataFrame in the dictionary
    monthly_dfs[month] = monthly_df



### Merge all hourly demand for each edge

In [72]:
# Initialize an empty list to store all the dataframes
all_dfs = []

# Iterate over each month and dataframe
for month, df in monthly_dfs.items():
    # Rename the columns to include the month as a suffix
    df = df.rename(columns=lambda x: f'{x}_{month}' if 'hour' in x else x)
    # Append the dataframe to the list
    all_dfs.append(df)

# Use merge to join all dataframes on 'u' and 'v'
from functools import reduce

merged_df = reduce(lambda  left,right: pd.merge(left, right, on=['u', 'v'], how='outer'), all_dfs)


### Find the highest demand value for each edge, it is equivalent to the needed supply in order to be classified as served

In [73]:
hourly_columns = [col for col in merged_df.columns if 'hour' in col]
merged_df['max_hourly_value'] = merged_df[hourly_columns].max(axis=1)


In [74]:
merged_df

,u,v,hour_00_jan_jan,max_hourly_value_jan,hour_01_jan_jan,hour_02_jan_jan,hour_03_jan_jan,hour_04_jan_jan,hour_05_jan_jan,hour_06_jan_jan,hour_07_jan_jan,hour_08_jan_jan,hour_09_jan_jan,hour_10_jan_jan,hour_11_jan_jan,hour_12_jan_jan,hour_13_jan_jan,hour_14_jan_jan,hour_15_jan_jan,hour_16_jan_jan,hour_17_jan_jan,hour_18_jan_jan,hour_19_jan_jan,hour_20_jan_jan,hour_21_jan_jan,hour_22_jan_jan,hour_23_jan_jan,hour_00_feb_feb,max_hourly_value_feb,hour_01_feb_feb,hour_02_feb_feb,hour_03_feb_feb,hour_04_feb_feb,hour_05_feb_feb,hour_06_feb_feb,hour_07_feb_feb,hour_08_feb_feb,hour_09_feb_feb,hour_10_feb_feb,hour_11_feb_feb,hour_12_feb_feb,hour_13_feb_feb,hour_14_feb_feb,hour_15_feb_feb,hour_16_feb_feb,hour_17_feb_feb,hour_18_feb_feb,hour_19_feb_feb,hour_20_feb_feb,hour_21_feb_feb,hour_22_feb_feb,hour_23_feb_feb,hour_00_mar_mar,max_hourly_value_mar,hour_01_mar_mar,hour_02_mar_mar,hour_03_mar_mar,hour_04_mar_mar,hour_05_mar_mar,hour_06_mar_mar,hour_07_mar_mar,hour_08_mar_mar,hour_09_mar_mar,hour_10_mar_mar,hour_11_mar_mar,hour_12_mar_mar,hour_13_mar_mar,hour_14_mar_mar,hour_15_mar_mar,hour_16_mar_mar,hour_17_mar_mar,hour_18_mar_mar,hour_19_mar_mar,hour_20_mar_mar,hour_21_mar_mar,hour_22_mar_mar,hour_23_mar_mar,hour_00_apr_apr,max_hourly_value_apr,hour_01_apr_apr,hour_02_apr_apr,hour_03_apr_apr,hour_04_apr_apr,hour_05_apr_apr,hour_06_apr_apr,hour_07_apr_apr,hour_08_apr_apr,hour_09_apr_apr,hour_10_apr_apr,hour_11_apr_apr,hour_12_apr_apr,hour_13_apr_apr,hour_14_apr_apr,hour_15_apr_apr,hour_16_apr_apr,hour_17_apr_apr,hour_18_apr_apr,hour_19_apr_apr,hour_20_apr_apr,hour_21_apr_apr,hour_22_apr_apr,hour_23_apr_apr,hour_00_may_may,max_hourly_value_may,hour_01_may_may,hour_02_may_may,hour_03_may_may,hour_04_may_may,hour_05_may_may,hour_06_may_may,hour_07_may_may,hour_08_may_may,hour_09_may_may,hour_10_may_may,hour_11_may_may,hour_12_may_may,hour_13_may_may,hour_14_may_may,hour_15_may_may,hour_16_may_may,hour_17_may_may,hour_18_may_may,hour_19_may_may,hour_20_may_may,hour_21_may_may,hour_22_may_may,hour_23_may_may,hour_00_jun_jun,max_hourly_value_jun,hour_01_jun_jun,hour_02_jun_jun,hour_03_jun_jun,hour_04_jun_jun,hour_05_jun_jun,hour_06_jun_jun,hour_07_jun_jun,hour_08_jun_jun,hour_09_jun_jun,hour_10_jun_jun,hour_11_jun_jun,hour_12_jun_jun,hour_13_jun_jun,hour_14_jun_jun,hour_15_jun_jun,hour_16_jun_jun,hour_17_jun_jun,hour_18_jun_jun,hour_19_jun_jun,hour_20_jun_jun,hour_21_jun_jun,hour_22_jun_jun,hour_23_jun_jun,hour_00_jul_jul,max_hourly_value_jul,hour_01_jul_jul,hour_02_jul_jul,hour_03_jul_jul,hour_04_jul_jul,hour_05_jul_jul,hour_06_jul_jul,hour_07_jul_jul,hour_08_jul_jul,hour_09_jul_jul,hour_10_jul_jul,hour_11_jul_jul,hour_12_jul_jul,hour_13_jul_jul,hour_14_jul_jul,hour_15_jul_jul,hour_16_jul_jul,hour_17_jul_jul,hour_18_jul_jul,hour_19_jul_jul,hour_20_jul_jul,hour_21_jul_jul,hour_22_jul_jul,hour_23_jul_jul,hour_00_aug_aug,max_hourly_value_aug,hour_01_aug_aug,hour_02_aug_aug,hour_03_aug_aug,hour_04_aug_aug,hour_05_aug_aug,hour_06_aug_aug,hour_07_aug_aug,hour_08_aug_aug,hour_09_aug_aug,hour_10_aug_aug,hour_11_aug_aug,hour_12_aug_aug,hour_13_aug_aug,hour_14_aug_aug,hour_15_aug_aug,hour_16_aug_aug,hour_17_aug_aug,hour_18_aug_aug,hour_19_aug_aug,hour_20_aug_aug,hour_21_aug_aug,hour_22_aug_aug,hour_23_aug_aug,hour_00_sep_sep,max_hourly_value_sep,hour_01_sep_sep,hour_02_sep_sep,hour_03_sep_sep,hour_04_sep_sep,hour_05_sep_sep,hour_06_sep_sep,hour_07_sep_sep,hour_08_sep_sep,hour_09_sep_sep,hour_10_sep_sep,hour_11_sep_sep,hour_12_sep_sep,hour_13_sep_sep,hour_14_sep_sep,hour_15_sep_sep,hour_16_sep_sep,hour_17_sep_sep,hour_18_sep_sep,hour_19_sep_sep,hour_20_sep_sep,hour_21_sep_sep,hour_22_sep_sep,hour_23_sep_sep,hour_00_oct_oct,max_hourly_value_oct,hour_01_oct_oct,hour_02_oct_oct,hour_03_oct_oct,hour_04_oct_oct,hour_05_oct_oct,hour_06_oct_oct,hour_07_oct_oct,hour_08_oct_oct,hour_09_oct_oct,hour_10_oct_oct,hour_11_oct_oct,hour_12_oct_oct,hour_13_oct_oct,hour_14_oct_oct,hour_15_oct_oct,hour_16_oct_oct,hour_17_oct_oct,hour_18_oct_oct,hour_19_oct_oct,hour_20_o

### Export the required supply for these edges

In [75]:
max_hourly_df = merged_df[['u', 'v', 'max_hourly_value']].copy()
max_hourly_df


,u,v,max_hourly_value
0,1499032,1512898,82.98891
1,954272,279457,2.11212
2,1306725,892553,34.49804
3,1306725,613853,36.52216
4,1306725,246612,22.88135
...,...,...,...
245,975503,975936,1.84811
246,975936,973638,1.14407
247,973638,973930,1.84811
248,832749,1101914,1.93611


In [76]:
#Recall the capabiilties of chargers

cars_per_hour_dict={
    350: 4.6,
    300: 4.0,
    250: 3.3,
    200: 2.6,
    180: 2.4,
    175: 2.3,
    150: 2.0,
    135: 1.8,
    129: 1.7,
    125: 1.7,
    120: 1.6,
    100: 1.3,
    75: 1.0,
    62.5: 0.8,
    62: 0.8,
    60: 0.8,
    55: 0.7,
    50: 0.7,
    48: 0.6,
    44: 0.6,
    43: 0.6
}




In [79]:

# We are imagining scenarios for how many chargers at 150kw or 50kw would be needed to 
# cover demand of all roads in Norway that currently do not have chargers.
# We are using 150 and 50kw examples as they are the most frequent types in the Norwegian charging infrastructure


# Constants representing the number of cars each charger type can serve in an hour
CARS_PER_HOUR_50KW = cars_per_hour_dict[50]
CARS_PER_HOUR_150KW = cars_per_hour_dict[150]

# Calculate number of each charger type needed to fulfill the max hourly volume
max_hourly_df["50kw_needed"] = np.ceil(max_hourly_df["max_hourly_value"] / CARS_PER_HOUR_50KW)
max_hourly_df["150kw_needed"] = np.ceil(max_hourly_df["max_hourly_value"] / CARS_PER_HOUR_150KW)

# Convert '50kw_needed' and '150kw_needed' to integer
max_hourly_df["50kw_needed"] = max_hourly_df["50kw_needed"].astype(int)
max_hourly_df["150kw_needed"] = max_hourly_df["150kw_needed"].astype(int)

# Calculate the total number of each type of charger needed
total_50kw_needed = max_hourly_df["50kw_needed"].sum()
total_150kw_needed = max_hourly_df["150kw_needed"].sum()

print("Total 50kw chargers needed: ", total_50kw_needed)
print("Total 150kw chargers needed: ", total_150kw_needed)



Total 50kw chargers needed:  2218
Total 150kw chargers needed:  855


In [80]:
max_hourly_df

,u,v,max_hourly_value,50kw_needed,150kw_needed
0,1499032,1512898,82.98891,119,42
1,954272,279457,2.11212,4,2
2,1306725,892553,34.49804,50,18
3,1306725,613853,36.52216,53,19
4,1306725,246612,22.88135,33,12
...,...,...,...,...,...
245,975503,975936,1.84811,3,1
246,975936,973638,1.14407,2,1
247,973638,973930,1.84811,3,1
248,832749,1101914,1.93611,3,1
